In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto", trust_remote_code=True)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

{"timestamp":"2025-07-17T08:36:05.538326Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(Connect, ConnectError(\"dns error\", Custom { kind: Uncategorized, error: \"failed to lookup address information: Temporary failure in name resolution\" })) }). Retrying..."},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":242}
{"timestamp":"2025-07-17T08:36:05.538400Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 1.116443106s before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}
{"timestamp":"2025-07-17T08:36:05.539393Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(Connect, ConnectError(\"dns error\", Custom { kind: Uncategorized, error: \"failed to lookup address information: Temporary failure

Cancellation requested; stopping current tasks.
Fetching 4 files:   0%|          | 0/4 [00:23<?, ?it/s]


KeyboardInterrupt: 

In [2]:
from neurofaith_main import neurofaith

device = "cuda"

nf = neurofaith(model, tokenizer, device)

In [20]:
import pandas as pd

df = pd.DataFrame({
    "text": [
        "The country of citizenship of the founder of Ebay is",
        "The location of formation of the publisher of Octopath Traveler is",
        "The country of citizenship of the screenwriter of Wings of Desire is",
        "The spouse of the performer of Imagine is",
    ],
    "e1_label": [  # e₁: starting entity mentioned in the question
        "Ebay",
        "Octopath Traveler",
        "Wings of Desire",
        "Imagine"
    ],
    "bridge_objects": [  # e₂: bridge entity that connects e₁ to e₃
        "Pierre Omidyar",
        "Nintendo",
        "Wim Wenders",
        "John Lennon",
    ],
    "gold_e3": [  # e₃: final answer
        "The United States",
        "Japan",
        "Germany",
        "Yoko Ono",
    ]
})

In [21]:
from utils import *

In [22]:
# Step 1: Predict the final answer (ô₃)
df["e3_answer"] = nf.answer_instruct(
    model=model,
    texts=df["text"].tolist(),
)

100%|██████████| 4/4 [00:00<00:00, 10.33it/s]


In [24]:
print(df["e3_answer"])

0             Paris
1     Paris, France
2             Paris
3    *Selena Gomez*
Name: e3_answer, dtype: object


In [25]:
df["e3_answer"] = clean_prediction(df["e3_answer"])

In [26]:
# Step 2: Ask for the self-NLE (e(x))
df["explanation"] = nf.self_explain(
    model=model,
    texts=df["text"].tolist(),
    answers=df["e3_answer"].tolist(),
)

In [27]:
df["explanation"] = clean_explanation(df["explanation"])

In [29]:
# Step 3: Extract the predicted bridge object (ô₂)
df["predicted_bridge_objects"] = nf.retrieve_bridge_object(
    retriever_model="Qwen/Qwen3-30B-A3B",
    texts=df["explanation"],
    e1_labels=df["e1_label"],
    e3_answers=df["e3_answer"],
    max_new_tokens=128,
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [11]:
df["predicted_bridge_objects"] = clean_bridge_objects(df["predicted_bridge_objects"])

In [12]:
# Step 4: Interpret the model's internals using Selfie
interpretation_result = nf.interpret_selfie(
    model=model,
    texts=df["text"],
    interpretation_prompt="What is the following? Answer briefly",
)

100%|██████████| 3/3 [00:40<00:00, 13.44s/it]


In [13]:
for k, v in interpretation_result.items():
    df[k] = v

In [14]:
col_interpretation = get_interpretation_columns([8,10,12], [3,4])

In [15]:
# Step 5: Compare explanation to internal activity (measure faithfulness)
df["faithful_NLE"] = nf.compute_faithfulness(
    data=df,
    predicted_bridge_objects_column="predicted_bridge_objects",
    col_interpretation=col_interpretation
)

In [16]:
# Final evaluations

df["prediction_status"] = get_prediction_status(df["gold_e3"], df["e3_answer"])

df["explanation_status"] = get_explanation_status(df["bridge_objects"], df["explanation"],
                                                  df["predicted_bridge_objects"])

In [17]:
pd.set_option('display.max_colwidth', None)     # don’t truncate strings
pd.set_option('display.max_columns', None)      # show all columns
pd.set_option('display.width', 1000)            # wide display

highlight = lambda x: 'color: red' if x is False else 'color: green'
df.style.applymap(highlight, subset=["prediction_status", "explanation_status", "faithful_NLE"])

/tmp/ipykernel_185603/3484351416.py:6: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(highlight, subset=["prediction_status", "explanation_status", "faithful_NLE"])


,text,e1_label,bridge_objects,gold_e3,e3_answer,explanation,predicted_bridge_objects,interpretation_8.3,interpretation_8.4,interpretation_10.3,interpretation_10.4,interpretation_12.3,interpretation_12.4,faithful_NLE,prediction_status,explanation_status
0,The country of citizenship of the founder of Ebay is,Ebay,Pierre Omidyar,The United States,Paris,"Sure, here's a simple explanation:The country of citizenship of the founder of eBay is Paris because he was born there and raised there.",Founder of eBay,"eBay is an online marketplace where people can buy and sell a wide variety of goods and services. It is similar to Amazon, but with a focus on a wider range of products. eBay was founded in 1995 by Pierre Omid","Ebay is an American multinational e-commerce company and online auction website. It is the world's largest online retailer, and its stock is listed on the New York Stock Exchange (NYSE: EBAY). Ebay was founded in 199","eBay is an online auction and retail website where people can buy and sell a wide variety of goods and services. It is one of the largest online retailers in the world, and it offers a wide range of products and services, including: *","eBay is a global online marketplace where people and businesses can buy and sell a wide variety of goods and services. It is one of the largest online retailers in the world, and it offers a wide range of products and services, including: *","eBay is an online auction and retail website where people can buy and sell a wide variety of goods and services. It is one of the largest online retailers in the world, and it offers a wide range of products and services, including: *","eBay is a global online marketplace for buying and selling a wide variety of goods and services. It is the largest online retailer in the world, and it has a wide range of products and services available. eBay is a subsidiary of PayPal, and",False,False,False
1,The location of formation of the publisher of Octopath Traveler is,Octopath Traveler,Nintendo,Japan,"Paris, France","The location of formation of the publisher of Octopath Traveler is in Paris, France.",Publisher,"2. At its core, what is the Traveler's purpose? To complete the Traveler's Quest, a quest that involves finding and defeating the Shadow Lord. The Traveler's journey takes them through a variety of worlds, each with","2e6. Answer Traveler is a tabletop role-playing game (TTRPG) in which players take on the roles of adventurers, heroes, and villains. The game is played with a set of dice and a set of rules.",2. Answer Traveler is a role-playing video game developed by BioWare and published by Microsoft. It is a prequel to the Elder Scrolls series of video games.,2. The Traveler's Handbook for the Traveler's Companion is the second book in the Traveler's Companion series of tabletop role-playing games.,2. Answer Traveler is a role-playing video game developed by BioWare and published by Microsoft. It is a prequel to the Dragon Age series of video games.,"2.0. The Traveler's Handbook, page 2.0, is a section of the game that provides information and guidance to players. It contains a wealth of knowledge about the game, including rules, strategies, and tips.",False,False,False
2,The country of citizenship of the screenwriter of Wings of Desire is,Wings of Desire,Wim Wenders,Germany,Paris,"Sure, the country of citizenship of the screenwriter of Wings of Desire is Paris. It is a city in France known for its rich culture and history.",Screenwriter,"The film ""The Devil's Desire"" is a romantic drama film released in 1988. The film stars George Clooney and Brad Pitt as two men who are drawn to each other despite their differences. The film explores the themes of","The film ""The Devil's Desire"" is a romantic drama film about a man named Jack Twist, a lonely and disillusioned artist, and a prostitute named Desiree. The film explores the themes of love, desire, and redemption.","1957. Answer Desire is a film noir film directed by Orson Welles and